In [162]:
from typing import List
import numpy as np
import pandas as pd
from pynwb import NWBFile, TimeSeries
from pynwb.behavior import BehavioralEvents
from pynwb.file import Subject
from pynwb.ecephys import ElectrodeGroup
import nixio
import regex as re
from usz_neuro_conversion.common import (
    SessionContext,
    NixContext,
    get_metadata_row,
    read_nix,
    get_date,
    write_nwb,
    standardize_sex,
    find_nix_files,
)

In [163]:
def convert_nix_to_nwb(subject: int, session: int) -> SessionContext:
    ctx = create_context(subject, session)
    write_subject(ctx)
    add_electrode_columns(ctx)
    ieeg_electrode_group = write_ieeg_electrodes(ctx)
    write_ieeg_measurements(ctx)
    write_trial_data(ctx)
    write_behavior(ctx)
    write_events(ctx)
    write_waveforms(ctx, ieeg_electrode_group)
    return ctx

In [164]:
def create_context(subject: int, session: int) -> SessionContext:
    nix_context = NixContext(subject, session, project="Human_MTL_units_visual_WM")
    nix = read_nix(nix_context)
    general = nix.sections["General"]
    nwb = NWBFile(
        session_description="Running experiment as described in the the experiment description",
        identifier=f"Human_MTL_units_visual_WM_subject{subject:02}_session{session:02}",
        session_start_time=get_date(nix_context),
        lab=general.props["Recording location"].values[0],
        institution="Universitätsspital Zürich, 8091 Zurich, Switzerland",  # Broken UTF-8 in file
        related_publications=_get_related_publications(nix),
        experimenter="Boran, Ece",
        experiment_description=_get_experiment(nix),
        keywords=[
            "Visual",
            "Spatial",
            "Neural decoding",
            "Hippocampus",
            "Entorhinal cortex",
        ],
    )
    return nix_context.to_session_context(nix, nwb)

In [165]:
def _get_experiment(nix: nixio.File) -> str:
    task = nix.sections["Task"].props
    task_name = task["Task name"].values[0]
    # Broken UTF-8 in file
    task_desc = "The task is a change detection task designed to examine the visual working memory of subjects. In each trial, arrays of colored squares were presented and had to be memorized. The number of squares determined the set size: 1, 2, 4 or 6. There was a total 192 trials per session. Each trial started with a warning signal (0.4 s) that was a red fixation dot. The fixation dot was then changed to black (0.4 – 0.5 s, jittered). A memory array (encoding period, 0.8 s) was followed by a delay (retention interval, 0.9 s). After the delay, a test array was shown (2 s) followed by a jittered inter-trial interval of 1.3 to 2.3 s. The participants indicated by button press (“Same” or “Different”, forced choice) whether the test array differed from the memory array. If the arrays differed, only one square changed in colour, but all squares remained on the same location. The fixation dot was visible on the screen during the whole trial period. Eight different colours were used for the memory and test array (yellow, red, green, blue, magenta, cyan, grey, black). Before starting the sessions, participants conducted trial runs in a practice session to learn the task. In this session we verified if participants were colour-blind and could discriminate all colours. Practice sessions were repeated until the participant understood the task and was able to follow the pace of the trials."
    task_url = "https://www.neurobs.com/ex_files/expt_view?id=285&tree_item_url=klaver12.exp&item_id=klaver12.exp"  # Found online
    return (
        f"Task Name: {task_name}\nTask Description: {task_desc}\nTask URL: {task_url}"
    )

In [166]:
def _get_related_publications(_nix: nixio.File) -> List[str]:
    dois = ["https://doi.org/10.1016/j.neuroimage.2022.119123"]  # Found online
    return [doi.strip() for doi in dois]

In [167]:
def write_subject(ctx: SessionContext):
    metadata = get_metadata_row(ctx.to_nix_context())
    age = metadata["Age"]
    sex = metadata["Sex"]
    ctx.nwb.subject = Subject(
        subject_id=f"{ctx.subject:02}",
        age=f"P{int(age)}Y",
        description=_get_subject_description(ctx),
        species="Homo sapiens",
        sex=standardize_sex(sex),
    )

In [168]:
def _get_subject_description(ctx: SessionContext) -> str:
    metadata = get_metadata_row(ctx.to_nix_context())
    subject = ctx.nix.sections["Subject"].props
    handedness = metadata["Handedness"]
    pathology = metadata["Pathology"]
    implanted_electrodes = subject["Implanted electrodes"].values[0]
    electrodes_in_soz = metadata["Electrodes in seizure onset zone (SOZ)"]
    return f"Handedness: {handedness}\nPathology: {pathology}\nImplanted electrodes: {implanted_electrodes}\nElectrodes in seizure onset zone (SOZ): {electrodes_in_soz}"

In [169]:
def add_electrode_columns(ctx: SessionContext):
    ctx.nwb.add_electrode_column(
        name="label",
        description="Channel label referenced by other data arrays",
    )
    ctx.nwb.add_electrode_column(
        name="is_inside_soz",
        description="Indicates whether the electrode is inside the seizure onset zone (SOZ)",
    )

In [170]:
def _get_session_data(ctx: SessionContext) -> nixio.Block:
    return ctx.nix.blocks[f"Data_Subject_{ctx.subject:02}_Session_{ctx.session:02}"]

In [171]:
def write_ieeg_electrodes(ctx: SessionContext) -> ElectrodeGroup:
    nwb = ctx.nwb

    device = nwb.create_device(
        name="ATLAS Neurophysiology System",
        manufacturer="Neuralynx, Inc.",
        description="iEEG recording system",
    )

    # create an electrode group for this group
    electrode_group = nwb.create_electrode_group(
        name="ieeg",
        description=f"iEEG electrodes",
        device=device,
        location="Intracranial",
    )

    electrodes = _get_ieeg_electrodes(ctx)
    electrodes.apply(
        lambda row: _add_row_to_ieeg_electrodes(nwb, electrode_group, row), axis=1
    )
    return electrode_group

In [172]:
VISUAL_TASK_ELECTRODES = pd.read_csv(
    "../out/metadata/visual_task_electrodes.tsv", sep="\t"
)
VISUAL_TASK_ELECTRODES["anatomical_location"] = VISUAL_TASK_ELECTRODES[
    "anatomical_location"
].fillna("unspecific")
VISUAL_TASK_ELECTRODES = VISUAL_TASK_ELECTRODES.astype(
    {"label": "string", "anatomical_location": "string", "inside_soz": "bool"}
)


def _get_ieeg_electrodes(ctx: SessionContext) -> pd.DataFrame:
    # only keep the current subject
    df = VISUAL_TASK_ELECTRODES.loc[
        VISUAL_TASK_ELECTRODES["participant"] == ctx.subject
        ]
    return df.reset_index()

In [173]:
def _get_ieeg_electrode_labels(ctx: SessionContext) -> List[str]:
    electrodes = _get_ieeg_electrodes(ctx)
    return electrodes["label"].tolist()

In [174]:
def _add_row_to_ieeg_electrodes(
        nwb: NWBFile, electrode_group: ElectrodeGroup, row: pd.Series
):
    # Got MNI map: +X is right, +Y is anterior, +Z is superior according to <https://kathleenhupfeld.com/mni-template-coordinate-systems/>
    # But need NWB: +X is posterior, +Y is inferior, +Z is right according to <https://pynwb.readthedocs.io/en/stable/pynwb.file.html#pynwb.file.NWBFile.add_electrode>
    nwb.add_electrode(
        group=electrode_group,
        label=row["label"],
        location=row["anatomical_location"],
        reference="Common intracranial reference",
        is_inside_soz=row["inside_soz"],
        x=-row["y"] if not np.isnan(row["y"]) else None,
        y=-row["z"] if not np.isnan(row["z"]) else None,
        z=row["x"] if not np.isnan(row["x"]) else None,
        filtering="Passband, 1 to 8000 Hz",
    )

In [175]:
def write_ieeg_measurements(ctx: SessionContext):
    nwb = ctx.nwb
    ieeg_electrode_indices = list(range(_get_ieeg_electrode_count(ctx)))
    nwb.create_electrode_table_region(
        region=ieeg_electrode_indices,  # reference row indices 0 to N-1
        description="ieeg electrodes",
    )

In [176]:
_IEEG_RE = re.compile(r"iEEG_Data_Trial_(\d+)")

In [177]:
def _get_ieeg_electrode_count(ctx: SessionContext) -> int:
    return len(_get_ieeg_electrode_labels(ctx))

In [178]:
def write_events(ctx: SessionContext):
    nwb = ctx.nwb
    session = _get_session_data(ctx)
    tags = session.groups["Trial events single tags spike times"].tags
    tags_by_trial = [(_EVENT_RE.findall(tag.name)[0], tag.position) for tag in tags]
    events = [
        (int(trial_number) - 1, name, position[0])
        for (name, trial_number), position in tags_by_trial
        if name != "Response"
    ]
    events.sort(key=lambda x: x[0])
    offset = _get_measurement_offset(ctx)
    duration = _get_trial_duration(ctx)
    events = [
        (name, time - offset + trial_number * duration)
        for trial_number, name, time in events
    ]
    events.append(("END", len(events) * duration))

    for (name, start), (_, end) in zip(events, events[1:]):
        nwb.add_epoch(
            start_time=start,
            stop_time=end,
            tags=name,
        )
        assert start < end

In [179]:
# Event_Retention_Trial_183_Spike_Times
_EVENT_RE = re.compile(r"Event_([a-zA-Z]+)_.*Trial_(\d+)_Spike_Times")

In [180]:
def write_trial_data(ctx: SessionContext):
    nwb = ctx.nwb
    nwb.add_trial_column(
        name="set_size",
        description="Number of colored squares in array presented during encoding period. Either 1, 2, 4, or 6.",
    )
    nwb.add_trial_column(
        name="solution",
        description='The correct answer to the question "Was the array presented during the test the same as the one presented during the encoding period?"',
    )

    nix = ctx.nix
    trials = nix.sections["Session"].sections["Trial properties"].sections
    duration = _get_trial_duration(ctx)
    for trial in trials:
        trial = trial.props
        trial_number = int(trial["Trial number"].values[0]) - 1
        start_time = trial_number * duration
        stop_time = start_time + duration
        nwb.add_trial(
            id=trial_number,
            start_time=start_time,
            stop_time=stop_time,
            set_size=int(trial["Set size"].values[0]),
            solution=trial["Match"].values[0] == 1,
        )

In [181]:
def write_behavior(ctx: SessionContext):
    nwb = ctx.nwb
    behavior_module = nwb.create_processing_module(
        name="behavior", description="Data for all trials in this session."
    )
    nix = ctx.nix
    trials = nix.sections["Session"].sections["Trial properties"].sections
    offset = _get_measurement_offset(ctx)
    duration = _get_trial_duration(ctx)
    data = []
    timestamps = []
    for trial in trials:
        trial_number = int(trial["Trial number"]) - 1
        trial = trial.props
        data.append(trial["Response"].values[0] == 1)
        time = trial["Response time"].values[0] - offset + trial_number * duration
        timestamps.append(time)
        # Could do this because everything after the response is fake time inbetween trials because NWB requires all trials to be on one long stretch
        # But not really necessary since we don't have any measurements then anyway
        # stop_time = (trial_number + 1.0) * duration
        # start_time = min(time, stop_time)
        # nwb.add_invalid_time_interval(
        #    start_time=start_time,
        #    stop_time=stop_time,
        # )

    time_series = TimeSeries(
        name="response",
        data=data,
        timestamps=timestamps,
        description='The participant\'s answer to the question "Was the array presented during the test the same as the one presented during the encoding period?"',
        unit="n/a",  # Might as well use https://github.com/rly/ndx-events, but it's not built-in...
        continuity="instantaneous",
    )

    behavioral_events = BehavioralEvents(
        name=f"BehavioralEvents.response", time_series=time_series
    )

    behavior_module.add(behavioral_events)

In [182]:
def write_waveforms(ctx: SessionContext, ieeg_electrode_group: ElectrodeGroup):
    nwb = ctx.nwb
    session = _get_session_data(ctx)
    waveforms = session.groups["Spike waveforms"].data_arrays
    spike_times = session.groups["Spike times"].data_arrays
    waveforms = [
        (_WAVEFORM_RE.findall(waveform.name)[0], waveform) for waveform in waveforms
    ]
    spike_times = [
        (_SPIKE_TIMES_RE.findall(spike_time.name)[0], spike_time[:])
        for spike_time in spike_times
    ]
    unit_to_waveform = {
        int(unit): (electrode, channel, values)
        for ((unit, electrode, channel), values) in waveforms
    }
    unit_to_trial_to_spike_times = {}
    for (unit, electrode, channel, trial), values in spike_times:
        unit_to_trial_to_spike_times.setdefault(int(unit), {})[trial] = (
            electrode,
            channel,
            values,
        )
    for unit, (electrode, channel, waveform) in unit_to_waveform.items():
        trial_to_spike_times = unit_to_trial_to_spike_times[unit]

        spike_times_for_trials = []
        for trial, (electrode_, channel_, spike_times) in trial_to_spike_times.items():
            assert electrode == electrode_
            assert channel == channel_
            spike_times_for_trials.append((trial, spike_times))
        spike_times_for_trials.sort(key=lambda x: x[0])
        spike_times_for_trials = [
            spike_times for _, spike_times in spike_times_for_trials
        ]
        spike_times_for_trials = _untrialize_irregular_timestamps(
            spike_times_for_trials, ctx
        )

        electrode_label = f"{electrode}{channel}"
        electrode_index = _get_electrode_index(ctx, electrode_label)
        # Don't see a way to include waveform sampling rate...
        offset = _get_measurement_offset(ctx) + waveform.dimensions[1].offset
        data = waveform[:]
        nwb.add_unit(
            id=int(unit),
            electrode_group=ieeg_electrode_group,
            electrodes=[electrode_index],
            waveform_mean=data[0] - offset,
            waveform_sd=data[1],
            spike_times=spike_times_for_trials,
        )

In [183]:
# Spike_Waveform_Unit_1_uAHL_2
_WAVEFORM_RE = re.compile(r"Spike_Waveform_Unit_(\d+)_u([a-zA-Z]+)_(\d+)")
# Spike_Times_Unit_36_uPHR_1_Trial_16
_SPIKE_TIMES_RE = re.compile(r"Spike_Times_Unit_(\d+)_u([a-zA-Z]+)_(\d+)_Trial_(\d+)")

In [184]:
def _untrialize_irregular_timestamps(
        timestamps: List[List[float]], ctx: SessionContext
) -> List[float]:
    offset = _get_measurement_offset(ctx)
    duration = _get_trial_duration(ctx)
    untrialized = []
    for trial, times in enumerate(timestamps):
        times = [time - offset + trial * duration for time in times]
        untrialized.extend(times)
    return untrialized

In [185]:
def _get_electrode_index(ctx: SessionContext, electrode: str) -> int:
    nwb = ctx.nwb
    index = next(
        index
        for index, electrodes in enumerate(nwb.electrodes["label"][:])
        if electrode in electrodes
    )
    return nwb.electrodes["id"][index]

In [186]:
def _get_trial_duration(ctx: SessionContext) -> float:
    offset = _get_measurement_offset(ctx)
    return ctx.nix.sections["Session"].props["Trial duration"].values[0] - offset

In [187]:
def _get_measurement_offset(_ctx: SessionContext) -> float:
    return -6.0  # Same as for verbal WM

Main

In [188]:
if __name__ == "__main__":
    context = convert_nix_to_nwb(10, 1)
    write_nwb(context)

In [189]:
if __name__ == "__main__":
    project = "Human_MTL_units_visual_WM"
    for subject, sessions in find_nix_files(project).items():
        for session, _ in sessions.items():
            print(f"Converting subject {subject} session {session}")
            try:
                context = convert_nix_to_nwb(subject, session)
                write_nwb(context)
                print("Done")
            except Exception as e:
                print(f"Failed to convert {subject} {session}")
                print(e)
    print("Everything done!")

Converting subject 1 session 1
Done
Converting subject 2 session 1
Done
Converting subject 3 session 1
Done
Converting subject 3 session 2
Done
Converting subject 4 session 1
Done
Converting subject 5 session 1
Done
Converting subject 6 session 1
Done
Converting subject 7 session 1
Done
Converting subject 8 session 1
Done
Converting subject 8 session 2
Done
Converting subject 9 session 1
Done
Converting subject 10 session 1
Done
Converting subject 11 session 1
Done
Converting subject 12 session 1
Done
Converting subject 12 session 2
Done
Converting subject 13 session 1
Done
Everything done!
